In [1]:
import datasets
from datasets import load_dataset

In [2]:
config = datasets.DownloadConfig(resume_download=True, max_retries=100)
dataset = load_dataset("gsm8k", "main", download_config=config)

test = dataset["test"]

In [3]:
from mindnlp.transformers import AutoModelForCausalLM, AutoTokenizer
from mindnlp.transformers.generation import GenerationConfig

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-7B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen1.5-7B").set_train(False)
model.generation_config = GenerationConfig.from_pretrained("Qwen/Qwen1.5-7B")
model.generation_config.do_sample = False

## Prompt Direct


In [4]:
q_lens = [len(d['question']) for d in dataset['train']]

In [5]:
import numpy as np
np.percentile(q_lens, [50, 80, 90, 95, 99, 100])

In [6]:
np.argmax(q_lens)

In [7]:
input_text = dataset['train'][3331]['question']

In [8]:
dataset['train'][3331]['answer']

In [9]:
prompt_direct = open('./lib_prompt/prompt_direct.txt').read()

In [10]:
prompt_q = prompt_direct + '\nQuestion: ' + input_text + '\n'

In [11]:
print(prompt_q)

In [17]:
input_ids = tokenizer.encode(prompt_q, return_tensors="ms")
input_ids.shape

In [18]:
outputs = model.generate(input_ids, max_new_tokens=256)
print(tokenizer.decode(outputs[0][input_ids.shape[1]:]))

## Complex Prompt

In [19]:
prompt_complex = open('./lib_prompt/prompt_complex.txt').read()

In [22]:
prompt_q = prompt_complex + '\nQuestion: ' + input_text + '\n'

In [23]:
print(prompt_q)

In [24]:
input_ids = tokenizer.encode(prompt_q, return_tensors="ms")
input_ids.shape

In [25]:
outputs = model.generate(input_ids, max_new_tokens=256)
print(tokenizer.decode(outputs[0][input_ids.shape[1]:]))